# About this notebook:
- We will be using articles IDs to collect each articles' information, including:
    - Title
    - Abstract
    - Given Mesh Terms
    - Publication Date
- This notebook is part of a series of notebooks under 'Step 2 Articles Collection', where each notebook contain articles published in a particular year.

# Import Libraries and Articles' IDs

In [1]:
# import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from Bio import Entrez
import xml.etree.ElementTree as ET

import time
from tqdm import tqdm

# Set the email address associated with your NCBI account
Entrez.email = "geok1723@gmail.com"

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


df=pd.read_csv("../Step_1_ID_Collection/IDs_2013.csv")

In [2]:
df.head()

,Date_str,ID,Date,Month,Year
0,2013/01/03,23278766,2013-01-03,1,2013
1,2013/01/30,23359179,2013-01-30,1,2013
2,2013/01/08,23289231,2013-01-08,1,2013
3,2013/01/10,23299705,2013-01-10,1,2013
4,2013/01/12,23305268,2013-01-12,1,2013


# Collecting Articles in groups of 100<br>
> The IDs are split into mini-dataframes containing 100 IDs for scraping. <br>
> Output will contain about 360 mini dataframes of articles that are published in this particular year. <br>
> This process was executed using Kaggle, and resulting dataframes are stored in the respective zip-folder. <br>

In [3]:
# determine the group number for each row
group_num = (df.index // 100) + 1

# map the group number to a group label (G1 to G6)
group_label = 'G' + group_num.astype(str)

# add the new column to the DataFrame
df['sub_group'] = group_label

# print(df)
# print(df['sub_group'].nunique())

df['Date']=pd.to_datetime(df['Date'],yearfirst=True)

# group the DataFrame by "Sub_group"
grouped = df.groupby('sub_group')

# create an empty dictionary to store the smaller DataFrames
df_dict = {}

# loop over the groups and create smaller DataFrames
for subgroup, subgroup_df in grouped:
    # construct the name of the smaller DataFrame
    year = subgroup_df['Year'].iloc[0]
    month = subgroup_df['Date'].iloc[0].strftime('%m')
    df_name = f"df_{year}_{month}_{subgroup}"
    # add the smaller DataFrame to the dictionary
    df_dict[df_name] = subgroup_df.copy()

df_ls = list(df_dict.keys())

df_ls.sort()

In [7]:
output_df = {}
for grp in tqdm(df_ls):
    _df = df_dict[grp]
    _s = _df[['ID','Date_str']] 

    print(f"Creating Dataframe of articles for {grp}")
    abstracts = []
    titles = []
    ids = []
    MeSH = []
    Pub_Date = []

    for index,row in tqdm(_s.iterrows()):	

        pub_date = row['Date_str']
        pmid = row['ID']


        try:
            handle = Entrez.efetch(db="pubmed", id=pmid, rettype="abstract", retmode="xml")

            xml_data = handle.read()
            handle.close()

            #obtain abstract and title
            root = ET.fromstring(xml_data)
            title = root.find(".//ArticleTitle").text.strip()
            abst = root.findall(".//AbstractText")

            abstract_text = ""
            for abstract in abst:
                abstract_text += abstract.text

            
            #obtain MeSH
            mesh_terms = ['']
            mesh_list = root.findall(".//MeshHeadingList/MeshHeading")
            for mesh in mesh_list:
                descriptor_name = mesh.find(".//DescriptorName").text
                qualifiers = mesh.findall(".//QualifierName")
                qualifier_names = [qualifier.text for qualifier in qualifiers]
                mesh_terms.append((descriptor_name, qualifier_names))
                
            MeSH.append(mesh_terms)            
            ids.append(pmid)
            abstracts.append(abstract_text)
            titles.append(title)
            Pub_Date.append(pub_date)

        except:
            print(f"Error here in ID{pmid}")

    # Store the data in a pandas dataframe
    data = {"ID": ids, "title": titles,"Pub_Date":Pub_Date, "abstract": abstracts, "MeSH_term":MeSH}
    data_df = pd.DataFrame(data)
    df_name = f"ABSTRACT{grp}"
    output_df[df_name] = data_df
    data_df.to_csv(f"{df_name}.csv", index=False)
    print(output_df.keys())

  0%|          | 0/360 [00:00<?, ?it/s]

Creating Dataframe of articles for df_2013_01_G1



0it [00:00, ?it/s]
1it [00:00,  5.86it/s]
2it [00:00,  2.84it/s]
3it [00:02,  1.02s/it]
4it [00:02,  1.46it/s]
5it [00:03,  1.74it/s]
6it [00:03,  1.98it/s]
7it [00:03,  2.17it/s]
8it [00:04,  2.33it/s]
9it [00:04,  2.39it/s]
10it [00:04,  2.52it/s]
11it [00:06,  1.09it/s]
12it [00:07,  1.45it/s]
13it [00:07,  1.65it/s]
14it [00:07,  1.80it/s]
15it [00:08,  2.11it/s]
16it [00:08,  2.24it/s]
17it [00:09,  2.35it/s]
18it [00:09,  2.52it/s]
19it [00:09,  2.54it/s]
20it [00:10,  2.62it/s]
21it [00:10,  2.65it/s]
22it [00:10,  2.60it/s]
23it [00:12,  1.43it/s]
24it [00:12,  1.75it/s]
25it [00:12,  2.10it/s]
26it [00:13,  2.01it/s]
27it [00:13,  1.99it/s]
28it [00:14,  2.49it/s]
29it [00:14,  2.47it/s]
30it [00:14,  2.45it/s]
31it [00:15,  2.69it/s]
32it [00:15,  2.66it/s]
33it [00:15,  2.59it/s]
34it [00:16,  2.77it/s]
35it [00:16,  2.72it/s]
36it [00:17,  2.67it/s]
37it [00:17,  2.69it/s]
38it [00:17,  2.75it/s]
39it [00:18,  2.73it/s]
40it [00:18,  2.68it/s]
41it [00:18,  2.63it/s]
42it 

Error here in ID25473713



89it [00:37,  2.57it/s]
90it [00:37,  2.72it/s]
91it [00:37,  2.70it/s]
92it [00:38,  2.71it/s]
93it [00:38,  2.77it/s]
94it [00:38,  2.68it/s]
95it [00:39,  2.66it/s]
96it [00:39,  2.69it/s]
97it [00:40,  2.71it/s]
98it [00:40,  2.76it/s]
99it [00:40,  2.69it/s]
100it [00:41,  2.43it/s]
  0%|          | 1/360 [00:41<4:06:05, 41.13s/it]

...

dict_keys(['ABSTRACTdf_2013_01_G1', 'ABSTRACTdf_2013_01_G10', 'ABSTRACTdf_2013_01_G11', 'ABSTRACTdf_2013_01_G12', 'ABSTRACTdf_2013_01_G13', 'ABSTRACTdf_2013_01_G14', 'ABSTRACTdf_2013_01_G15', 'ABSTRACTdf_2013_01_G16', 'ABSTRACTdf_2013_01_G17', 'ABSTRACTdf_2013_01_G18', 'ABSTRACTdf_2013_01_G19', 'ABSTRACTdf_2013_01_G2', 'ABSTRACTdf_2013_01_G20', 'ABSTRACTdf_2013_01_G21', 'ABSTRACTdf_2013_01_G22', 'ABSTRACTdf_2013_01_G23', 'ABSTRACTdf_2013_01_G24', 'ABSTRACTdf_2013_01_G25', 'ABSTRACTdf_2013_01_G26', 'ABSTRACTdf_2013_01_G27', 'ABSTRACTdf_2013_01_G28', 'ABSTRACTdf_2013_01_G29', 'ABSTRACTdf_2013_01_G3', 'ABSTRACTdf_2013_01_G30', 'ABSTRACTdf_2013_01_G4', 'ABSTRACTdf_2013_01_G5', 'ABSTRACTdf_2013_01_G6', 'ABSTRACTdf_2013_01_G7', 'ABSTRACTdf_2013_01_G8', 'ABSTRACTdf_2013_01_G9', 'ABSTRACTdf_2013_02_G31', 'ABSTRACTdf_2013_02_G32', 'ABSTRACTdf_2013_02_G33', 'ABSTRACTdf_2013_02_G34', 'ABSTRACTdf_2013_02_G35', 'ABSTRACTdf_2013_02_G36', 'ABSTRACTdf_2013_02_G37', 'ABSTRACTdf_2013_02_G38', 'ABSTRACTd


0it [00:00, ?it/s]
1it [00:00,  2.80it/s]
2it [00:00,  3.14it/s]
3it [00:01,  2.92it/s]
4it [00:01,  2.74it/s]
5it [00:01,  2.81it/s]
6it [00:02,  2.71it/s]

Error here in ID24349761



7it [00:02,  2.68it/s]
8it [00:02,  2.77it/s]
9it [00:03,  2.75it/s]
10it [00:03,  2.76it/s]
11it [00:03,  2.73it/s]
12it [00:04,  2.68it/s]
13it [00:04,  2.61it/s]
14it [00:05,  2.75it/s]
15it [00:05,  2.73it/s]
16it [00:05,  2.73it/s]
17it [00:06,  2.61it/s]
18it [00:06,  2.75it/s]
19it [00:07,  2.56it/s]
20it [00:07,  2.75it/s]
21it [00:07,  2.73it/s]
22it [00:08,  2.74it/s]
23it [00:08,  2.73it/s]
24it [00:08,  2.73it/s]
25it [00:09,  2.74it/s]
26it [00:09,  2.53it/s]
27it [00:09,  2.71it/s]
28it [00:10,  2.74it/s]
29it [00:10,  2.77it/s]
30it [00:11,  2.67it/s]
31it [00:11,  2.77it/s]
32it [00:11,  2.75it/s]
33it [00:12,  2.72it/s]
34it [00:12,  2.72it/s]
35it [00:12,  2.62it/s]
36it [00:13,  2.68it/s]
37it [00:13,  2.67it/s]
38it [00:13,  2.69it/s]
39it [00:14,  2.71it/s]
40it [00:15,  1.94it/s]
41it [00:15,  2.43it/s]
42it [00:15,  2.52it/s]
43it [00:16,  2.57it/s]
44it [00:16,  2.56it/s]
45it [00:16,  2.59it/s]
46it [00:17,  2.62it/s]
47it [00:17,  2.67it/s]
48it [00:17,  2.65

dict_keys(['ABSTRACTdf_2013_01_G1', 'ABSTRACTdf_2013_01_G10', 'ABSTRACTdf_2013_01_G11', 'ABSTRACTdf_2013_01_G12', 'ABSTRACTdf_2013_01_G13', 'ABSTRACTdf_2013_01_G14', 'ABSTRACTdf_2013_01_G15', 'ABSTRACTdf_2013_01_G16', 'ABSTRACTdf_2013_01_G17', 'ABSTRACTdf_2013_01_G18', 'ABSTRACTdf_2013_01_G19', 'ABSTRACTdf_2013_01_G2', 'ABSTRACTdf_2013_01_G20', 'ABSTRACTdf_2013_01_G21', 'ABSTRACTdf_2013_01_G22', 'ABSTRACTdf_2013_01_G23', 'ABSTRACTdf_2013_01_G24', 'ABSTRACTdf_2013_01_G25', 'ABSTRACTdf_2013_01_G26', 'ABSTRACTdf_2013_01_G27', 'ABSTRACTdf_2013_01_G28', 'ABSTRACTdf_2013_01_G29', 'ABSTRACTdf_2013_01_G3', 'ABSTRACTdf_2013_01_G30', 'ABSTRACTdf_2013_01_G4', 'ABSTRACTdf_2013_01_G5', 'ABSTRACTdf_2013_01_G6', 'ABSTRACTdf_2013_01_G7', 'ABSTRACTdf_2013_01_G8', 'ABSTRACTdf_2013_01_G9', 'ABSTRACTdf_2013_02_G31', 'ABSTRACTdf_2013_02_G32', 'ABSTRACTdf_2013_02_G33', 'ABSTRACTdf_2013_02_G34', 'ABSTRACTdf_2013_02_G35', 'ABSTRACTdf_2013_02_G36', 'ABSTRACTdf_2013_02_G37', 'ABSTRACTdf_2013_02_G38', 'ABSTRACTd